In [ ]:
import numpy as np
import scipy.signal as signal
import plotly.express as px
import pandas as pd # pour les dataframes
import sounddevice as sd # pour jouer les sons associés aux signaux

ok


## 1 Fenêtrage temporel
1. Décrire le fonctionnement de la fonction np.fft.fftshift() , et son intérêt pour l’analyse spectrale. Dé-
terminer la transformation à appliquer sur l’axe fréquentiel pour afficher de manière cohérente le spectre obtenu
après l’application de la fonction np.fft.fftshift() .
--> Elle permet de mettre les BF au centre

In [4]:
help(np.fft.fftshift)

Help on _ArrayFunctionDispatcher in module numpy.fft:

fftshift(x, axes=None)
    Shift the zero-frequency component to the center of the spectrum.

    This function swaps half-spaces for all axes listed (defaults to all).
    Note that ``y[0]`` is the Nyquist component only if ``len(x)`` is even.

    Parameters
    ----------
    x : array_like
        Input array.
    axes : int or shape tuple, optional
        Axes over which to shift.  Default is None, which shifts all axes.

    Returns
    -------
    y : ndarray
        The shifted array.

    See Also
    --------
    ifftshift : The inverse of `fftshift`.

    Examples
    --------
    >>> import numpy as np
    >>> freqs = np.fft.fftfreq(10, 0.1)
    >>> freqs
    array([ 0.,  1.,  2., ..., -3., -2., -1.])
    >>> np.fft.fftshift(freqs)
    array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])

    Shift the zero-frequency component only along the second axis:

    >>> freqs = np.fft.fftfreq(9, d=1./9).reshape(3, 3)
   

2. Grâce aux fonctions signal.windows.hann() et signal.windows.boxcar() , créer deux sé-
quences wH et wB composées de K = 32 points correspondant respectivement aux fenêtres de Hann et rectan-
gulaire.
Afficher ces deux fenêtres en fonction du temps.

In [19]:
K = 32
t = np.linspace(0,K,  32)

wH = signal.windows.hann(K)
wB = signal.windows.boxcar(K)

#On affiche les 2 sinaux
fig1 = px.line(x=t, y=wH, title='sequence wH', labels={'x':'t', 'y':'fx'}).show()
fig1 = px.line(x=t, y=wB, title='sequence wB', labels={'x':'t', 'y':'fx'}).show()

3. Calculer la valeur moyenne des 2 fenêtres précédentes ( np.mean() ). En déduire G, le facteur d’amplitude à
appliquer sur wH pour que les deux fenêtres aient la même moyenne